# Samples for new CI template code

This is a sample notebook to test new CI template using message exchange.

## Description

This is a new template code for language-agnostic worker samples.

## Sample 0: Boilarplate code for calling CI services

### Find your server IP address first!

In [1]:
IP = '192.168.99.100'
BASE = 'http://' + IP + '/v1/'  # You need to change this to your service server

In [2]:
import requests
import json

def jprint(data):
    print(json.dumps(data, indent=4))

# Change this to  your Docker container's IP
HEADERS = {'Content-Type': 'application/json'}

# Check API Status
res = requests.get(BASE + 'services')
print("Here is the list of services supported by this submit agent:")
jprint(res.json())

# List of sample file locations
small_network = 'https://s3-us-west-2.amazonaws.com/ci-service-data/small.sif'
med_network = 'https://s3-us-west-2.amazonaws.com/ci-service-data/yeastHighQuality.sif'

# Pre-computed kernel and network (HumanNet)
human_net = 'http://chianti.ucsd.edu/~kono/ci/data/HumanNet.v1.symbol.sif'
human_net_kernel = 'http://chianti.ucsd.edu/~kono/ci/data/human_net.kernel.symbol.txt'

Here is the list of services supported by this submit agent:
[
    {
        "serviceName": "subnet", 
        "portNumber": "5557"
    }, 
    {
        "serviceName": "kernel", 
        "portNumber": "5556"
    }
]


## Sample1: Find Subnetwork using pre-computed kernel file

if you already have pre-computed kernel file, you can use it to find sub networks using:

* URL of Kernel file
* URL of  SIF (network) file

### _f (kernel, network, query) = subnetwork_

In [3]:
# Pre-computed sample kernel and network (HumanNet) stored in UCSD web server (about 0.5 GB)
human_net = 'http://chianti.ucsd.edu/~kono/ci/data/HumanNet.v1.symbol.sif'
human_net_kernel = 'http://chianti.ucsd.edu/~kono/ci/data/human_net.kernel.symbol.txt'

# Specify kernel and SIF file locations
base_query = {
    'kernel_url': human_net_kernel,
    'network_url': human_net,
}

gene_list = ['BRCA1', 'MAPK1']
base_query['query'] = gene_list

res = requests.post(BASE + 'services/subnet', data=json.dumps(base_query), headers=HEADERS)
job_id1 = res.json()['job_id']
jprint(res.json())

{
    "status": "queued", 
    "job_id": "8e6cf61b-a396-4a3e-b050-fe5dab3a9407"
}


In [5]:
# Check the status of the job
res = requests.get(BASE + 'queue')
jprint(res.json())

[
    {
        "status": "finished", 
        "job_id": "8e6cf61b-a396-4a3e-b050-fe5dab3a9407"
    }
]


In [6]:
# Get the result (Subnetwork in CX format)
result_url = BASE + 'queue/' + job_id1 + '/result'
print(result_url)
res = requests.get(result_url)
#jprint(res.json())

http://192.168.99.100/v1/queue/8e6cf61b-a396-4a3e-b050-fe5dab3a9407/result


## Sample 2: Create kernel from SIF file

### _f (network) = kernel_

In [7]:
# Specify locations of the SIF files
sif_file_locations = [small_network, med_network]

kernel_generation_tasks = {}

for sif_location in sif_file_locations:
    body = {
        'network_url': sif_location
    }
    res = requests.post(BASE + 'services/kernel', json=body, headers=HEADERS)
    kernel_generation_tasks[sif_location] = res.json()['job_id']
    jprint(res.json())

{
    "status": "queued", 
    "job_id": "5913c942-af02-41c7-ad2d-449172783f4a"
}
{
    "status": "queued", 
    "job_id": "5c34e020-b308-42c3-ad69-682452c92948"
}


In [8]:
# Check the status of the job
res = requests.get(BASE + 'queue')
jprint(res.json())

[
    {
        "status": "finished", 
        "job_id": "5c34e020-b308-42c3-ad69-682452c92948"
    }, 
    {
        "status": "finished", 
        "job_id": "8e6cf61b-a396-4a3e-b050-fe5dab3a9407"
    }, 
    {
        "status": "finished", 
        "job_id": "5913c942-af02-41c7-ad2d-449172783f4a"
    }
]


In [9]:
# Get result
job_id_for_small_network = kernel_generation_tasks[small_network]

kernel_url = BASE + 'queue/' + job_id_for_small_network + '/result'
print(kernel_url)
res = requests.get(kernel_url)
kernel_info = res.json()
jprint(kernel_info)

http://192.168.99.100/v1/queue/5913c942-af02-41c7-ad2d-449172783f4a/result
{
    "kernel_id": "bb855967-2217-4b6c-8ac9-c52a81f0901e", 
    "kernel_file": "http://kernelserver:3000/data/fba36f30-6178-11e5-b4c9-11f010adaf04", 
    "network": "https://s3-us-west-2.amazonaws.com/ci-service-data/small.sif"
}


In [10]:
# Use the result to find subnetwork
base_query = {
    'kernel_url': kernel_info['kernel_file'],
    'network_url': kernel_info['network']
}

gene_list = ["NRAS", "KRAS", "MAPK1"]
base_query['query'] = gene_list

res = requests.post(BASE + 'services/subnet', data=json.dumps(base_query), headers=HEADERS)
find_subnet_job_id = res.json()['job_id']
jprint(res.json())

{
    "status": "queued", 
    "job_id": "f76de101-5abb-489d-b79c-648beedc1096"
}


In [11]:
# Check the result
result_url = BASE + 'queue/' + find_subnet_job_id + '/result'
print(result_url)
res = requests.get(result_url)
#jprint(res.json())

http://192.168.99.100/v1/queue/f76de101-5abb-489d-b79c-648beedc1096/result


## Sample 3: Get actual Kernel file
For current configuration, kernels will be saved in a server in a restricted zone (= docker container separated from internet zone).  To access this, you need to open port. 

__Of course, this may be changed. It depends on deployment target.__


In [12]:
import pandas as pd

temp = kernel_info['kernel_file']
parts = temp.split('/')
parts[2] = IP + ':3333' # This is the default opened port...
kernel_url = '/'.join(parts)
print(kernel_url)

df = pd.read_csv(kernel_url, sep="\t")
df.head()

http://192.168.99.100:3333/data/fba36f30-6178-11e5-b4c9-11f010adaf04


,Key,AASDHPPT,ABL1,ACP1,ADI1,ADIPOQ,ADORA2A,ADRA2A,ADRA2B,ADRB2,...,UBTF,UCN,USF1,VASP,VAV1,VEGFA,WEE1,XIAP,YES1,ZFP36
0,AASDHPPT,0.905697,0.000005,0.000005,0.000860,0.000005,0.000860,0.000005,0.000005,0.000005,...,0.000005,0.000003,0.000005,0.000860,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005
1,ABL1,0.000005,0.905089,0.000252,0.000005,0.000252,0.000005,0.000252,0.000252,0.000252,...,0.000252,0.000008,0.000252,0.000005,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252
2,ACP1,0.000005,0.000252,0.905089,0.000005,0.000252,0.000005,0.000252,0.000252,0.000252,...,0.000252,0.000008,0.000252,0.000005,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252
3,ADI1,0.000860,0.000005,0.000005,0.905697,0.000005,0.000860,0.000005,0.000005,0.000005,...,0.000005,0.000003,0.000005,0.000860,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005
4,ADIPOQ,0.000005,0.000252,0.000252,0.000005,0.905089,0.000005,0.000252,0.000252,0.000252,...,0.000252,0.000008,0.000252,0.000005,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252
